In [ ]:
# 🚀 To get started, install DeepInverse by creating a new cell and running `%pip install deepinv`


# Using state-of-the-art diffusion models from HuggingFace Diffusers with DeepInverse

This demo shows you how to use our wrapper
:class:`deepinv.models.DiffusersDenoiserWrapper` to turn any SOTA models from the HuggingFace Hub to an image denoiser. It also can be used to perform unconditional image generation or for posterior sampling.

See more about the [diffusers pipeline](https://huggingface.co/docs/diffusers/index) and our posterior sampling [user guide](https://deepinv.github.io/deepinv/auto_examples/sampling/demo_diffusion_sde.html).

<div class="alert alert-info"><h4>Note</h4><p>This example requires the `diffusers` and `transformers` package. You can install it via `pip install diffusers transformers`.</p></div>


In [ ]:
import torch
import deepinv as dinv
from deepinv.models.wrapper import DiffusersDenoiserWrapper

device = dinv.utils.get_device()
dtype = torch.float32
figsize = 2.5

from deepinv.sampling import PosteriorDiffusion, EulerSolver, VarianceExplodingDiffusion
from deepinv.optim import ZeroFidelity

----------------------------------------------------

Let us first load a pretrained diffusion model from the HuggingFace Hub. Here, we use the `google/ddpm-ema-celebahq-256` model.
This model is trained on 256x256 of CelebA dataset using the DDPM scheduler.



In [ ]:
# We can wrap any diffusers model as a DeepInv denoiser using one line of code:
denoiser = DiffusersDenoiserWrapper(
    mode_id="google/ddpm-ema-celebahq-256", device=device
)

# Load an example image
x = dinv.utils.load_example(
    "celeba_example2.jpg",
    img_size=256,
    resize_mode="resize",
).to(device)

# Add noise and test the denoiser
sigma = 0.1
x_noisy = x + sigma * torch.randn_like(x)
with torch.no_grad():
    x_denoised = denoiser(x_noisy, sigma=sigma)

dinv.utils.plot(
    [x, x_noisy, x_denoised],
    figsize=(figsize * 3, figsize),
    titles=["Original image", "Noisy image", "Denoised image"],
)

---------------------------------

It is also possible to use the wrapped model for unconditional image generation.
The model was trained with DDPM scheduler, however we can use it with any SDE provided in DeepInv.
Here, we use the Variance Exploding SDE with Euler solver for sampling.



In [ ]:
num_steps = 125
rng = torch.Generator(device)
timesteps = torch.linspace(1, 0.001, num_steps)
solver = EulerSolver(timesteps=timesteps, rng=rng)

sde = VarianceExplodingDiffusion(
    device=device,
    dtype=dtype,
)

model = PosteriorDiffusion(
    data_fidelity=ZeroFidelity(),
    sde=sde,
    denoiser=denoiser,
    solver=solver,
    dtype=dtype,
    device=device,
    verbose=True,
)


sample, trajectory = model(
    y=None,
    physics=None,
    x_init=(1, 3, 256, 256),
    seed=42,
    get_trajectory=True,
)
dinv.utils.plot(
    sample,
    titles="Unconditional generation",
    figsize=(figsize, figsize),
)

---------------------

Similar to other denoisers in DeepInv, the wrapped diffusers model can be used for posterior sampling.
Below we use the same VE-SDE for posterior sampling in an inpainting problem.



In [ ]:
# Initialize the physics

mask = torch.ones_like(x)
mask[..., 70:150, 120:180] = 0
physics = dinv.physics.Inpainting(
    mask=mask,
    img_size=x.shape[1:],
    device=device,
    noise_model=dinv.physics.GaussianNoise(0.05),
)

y = physics(x)

In [ ]:
from deepinv.sampling import DPSDataFidelity

model = PosteriorDiffusion(
    data_fidelity=DPSDataFidelity(denoiser=denoiser, weight=1.0),
    denoiser=denoiser,
    sde=sde,
    solver=solver,
    dtype=dtype,
    device=device,
    verbose=True,
)

In [ ]:
posterior_sample = model(
    y=y,
    physics=physics,
    x_init=(1, 3, 256, 256),
    seed=15,
)
dinv.utils.plot(
    [x, y, posterior_sample],
    titles=["Original image", "Measurement", "Posterior sample"],
    figsize=(figsize * 3, figsize),
)